In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src import *
from model.classification import *

from tqdm.notebook import tqdm
from easydict import EasyDict
from IPython.display import HTML
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['agg.path.chunksize'] = 10000

def show_video(file):
    return HTML(f"""
        <video width="{1920 * 0.3}" height="{1080 * 0.3}" controls>
            <source src="data/video/{file}.MP4" type="video/mp4">
        </video>
    """)

params = EasyDict()
params.n_fft = 1024
params.n_mels = 64
params.hop_length = 128
params.sr = 44100
predictions = None

In [ ]:
# file = '20190819-Kutna Hora-L4-out-MVI_0040'
file = '20190819-Kutna Hora-L3-out-MVI_0008'

signal = load_audio(f'data/audio/{file}.MP4.wav')
csv = load_csv(f'{file}.MP4')
events = load_events_from_csv(csv)
directions = load_directions_from_csv(csv)
views = load_views_from_csv(csv)

try:
    manual_events = load_events(f'data/labels/{file}.MP4.txt')
except:
    manual_events = None

events_start_time, events_end_time = load_event_time_from_csv(csv)

In [ ]:
uuid = 1636031305
suffix = 'interval'
# from_time, till_time = 0, 60 * 10
from_time, till_time = 1560, 2064


model, params = load_model(uuid, suffix=suffix)
transform = create_transformation(params)
predictions, probabilities = validate(signal, model, transform, params, from_time=from_time, till_time=till_time, return_probs=True)

In [ ]:
show(
    params,
    signal=signal,
    events=events, 
    predictions=predictions,
    probabilities=probabilities,
    events_start_time=events_start_time,
    events_end_time=events_end_time,
    manual_events=manual_events,
    views=views,
    from_time=from_time,
    till_time=till_time,
    save=False
)

show_video(file)

In [ ]:
show_video('20190819-Kutna Hora-L10-in-MVI_0029')

### Visualize all videos

In [ ]:
files = [
    '20190819-Kutna Hora-L1-out-MVI_0007',
    '20190819-Kutna Hora-L3-in-MVI_0005',
    '20190819-Kutna Hora-L3-out-MVI_0008',
    '20190819-Kutna Hora-L4-in-MVI_0013',
    '20190819-Kutna Hora-L5-in-MVI_0003',
    '20190819-Kutna Hora-L6-out-MVI_0017',
    '20190819-Kutna Hora-L7-out-MVI_0032',
    '20190819-Kutna Hora-L8-in-MVI_0045',
    '20190819-Kutna Hora-L9-in-MVI_0043',
    '20190819-Kutna Hora-L10-out-SDV_1888',
    '20190819-Kutna Hora-L13-in-MVI_0006',
    '20190819-Kutna Hora-L13-out-MVI_0018',
    '20190819-Kutna Hora-L14-out-MVI_0005',
    '20190819-Kutna Hora-L15-out-MVI_0012',
    '20190819-Kutna Hora-L16-out-MVI_0003',
    '20190819-Kutna Hora-L18-in-MVI_0030',
    '20190819-Ricany-L2-in-MVI_0006',
    '20190819-Ricany-L2-out-MVI_0005',
    '20190819-Ricany-L3-in-MVI_0006',
    '20190819-Ricany-L6-in-MVI_0008',
    '20190819-Ricany-L6-out-MVI_0011',
    '20190819-Ricany-L7-in-MVI_0008',
    '20190819-Ricany-L7-out-MVI_0013',
    '20190819-Ricany-L8-in-MVI_0009',
    '20190819-Ricany-L8-out-MVI_0013',
    '20190819-Ricany-L9-in-MVI_0008',
    '20190819-Ricany-L9-out-MVI_0011'
]

window_length = 6
split_ratio = 0.75

datapool = DataPool(files, window_length, split_ratio, tqdm=tqdm)

In [ ]:
uuid = 1635061611
suffix = 'interval'

# for file in tqdm(files):
for video in tqdm(datapool):
    file = video.file
    signal = video.signal
    csv = load_csv(f'{file}.MP4')
    events = load_events_from_csv(csv)
    directions = load_directions_from_csv(csv)
    views = load_views_from_csv(csv)
    
    print(file)

    try:
        manual_events = load_events(f'data/labels/{file}.MP4.txt')
    except:
        manual_events = None

    events_start_time, events_end_time = load_event_time_from_csv(csv)
    
    model, params = load_model(uuid, suffix=suffix)
    transform = create_transformation(params)
    
    is_trn = True
    from_time, till_time = video.get_from_till_time(is_trn)
    from_time = int(from_time)
    till_time = int(till_time)
    predictions, probabilities = validate(signal, model, transform, params, from_time=from_time, till_time=till_time, return_probs=True)
    
    show(
        params,
        signal=signal,
        events=events, 
        predictions=predictions,
        probabilities=probabilities,
        events_start_time=events_start_time,
        events_end_time=events_end_time,
        manual_events=manual_events,
        views=views,
        from_time=from_time,
        till_time=till_time,
        save=False
    )
    
    is_trn = False
    from_time, till_time = video.get_from_till_time(is_trn)
    from_time = int(from_time)
    till_time = int(till_time)
    predictions, probabilities = validate(signal, model, transform, params, from_time=from_time, till_time=till_time, return_probs=True)
    plt.close('all')
        
    show(
        params,
        signal=signal,
        events=events, 
        predictions=predictions,
        probabilities=probabilities,
        events_start_time=events_start_time,
        events_end_time=events_end_time,
        manual_events=manual_events,
        views=views,
        from_time=from_time,
        till_time=till_time,
        save=False
    )
    
    plt.close('all')